In [5]:
import requests
import pandas as pd


In [9]:
import requests

def obter_dados_da_api(url):
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao conectar com a API: {e}")
        return None

url = "https://restcountries.com/v3.1/all"
dados = obter_dados_da_api(url)

if dados:
    print("Conexão bem-sucedida! Dados obtidos:")
    print(dados)
else:
    print("Não foi possível obter os dados da API.")


Erro ao conectar com a API: ('Connection broken: IncompleteRead(32534 bytes read, 138644 more expected)', IncompleteRead(32534 bytes read, 138644 more expected))
Não foi possível obter os dados da API.


In [10]:
if data is not None:
    print("Dados recebidos:")
    print(data[0])
else:
    print("Falha ao acessar dados do site.")



Falha ao acessar dados do site.


In [11]:
def criar_dataframe(data):
    
    if data is not None:
        return pd.DataFrame([{
            "Nome Comum": country["name"]["common"],
            "Nome Oficial": country["name"]["official"],
            "Código Alpha-2": country["cca2"],
            "Região": country["region"],
            "Latitude": country["latlng"][0] if country.get("latlng") else None,
            "Longitude": country["latlng"][1] if country.get("latlng") else None,
            "Capital": country["capital"][0] if country.get("capital") else None
        } for country in data])
    else:
        print("Falha ao acessar dados do site.")
        return None

In [12]:
def criar_dataframe_populacao(data):
   
    if data is not None:
        return pd.DataFrame([{
            "Nome Comum": country["name"]["common"],
            "População": country["population"],
            "Área": country["area"],
            "Continente": country["continents"][0] if country.get("continents") else None,
            "País sem litoral": country["landlocked"]
        } for country in data])
    else:
        print("Falha ao acessar dados do site.")
        return None

In [13]:
def criar_dataframe_membro_da_onu(data):

    if data is not None:
        return pd.DataFrame([{
            "Nome Comum": country["name"]["common"],
            "Independente": country.get("independent"),
            "Membro da ONU": country.get("unMember"),
            "Status": country.get("status")
        } for country in data])
    else:
        print("Falha ao acessar dados do site.")
        return None